#Import

Libraries are usually imported in the beginning.

In [0]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms
import torchvision.transforms.functional as FT

import matplotlib.pyplot as plt 

# GPU Check

You can usually check the version of most libraries by calling the `__version__` attribute.


In [0]:
torch.__version__

You can check if the GPU is being used by running the following command.

In [0]:
torch.cuda.is_available()

Use the following line of code whenever writing deep learning code, since it gives flexibility to the user.

In [0]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Tensor Basics

## Defining Tensors

You can initialize tensors in many ways. The most common ones are `.tensor()`, `.rand()` and `.zeros()`.

In [0]:
x = torch.empty(5, 3)
print(x)
print(x.size())

In [0]:
y = torch.rand(5, 3)
print(y)
print(y.shape)

In [0]:
x = torch.tensor([1, 2, 3])
print(x)

In [0]:
y = torch.rand_like(x, dtype=torch.float32)
print(y)

In [0]:
y.shape

In [0]:
z = torch.zeros((5, 3), dtype=torch.float32, device=device)
print(z)

## Tensor Operations

You can usually perform operations on torch tensors in more ways than 1.

In [0]:
print(x)

In [0]:
print(y)

In [0]:
z1 = x + y
print(z1)

In [0]:
z2 = torch.add(x, y)
print(z2)

You can either perform `out-place` operations:

In [0]:
r1 = torch.Tensor(2,3)
torch.add(x, y, out=r1)

Or can perform `in-place` operations:


In [0]:
y.add_(x)
print(y)

There is a difference between `x + y` and `x+=y`.

In [0]:
print(id(x))

x = x + y
print(id(x))

x += y
print(id(x))

## Reshaping Tensors

In [0]:
x = torch.randn(4, 4)
y = x.view(16)
z = x.view(-1, 8)  # the size -1 is inferred from other dimensions

print(x.size(), y.size(), z.size())

## Torch tensors to NumPy arrays and back

In [0]:
x = torch.rand((3, 3))
print(x)
print(type(x))

In [0]:
y = x.numpy()
print(y)
print(type(y))

In [0]:
z = torch.from_numpy(y)
print(z)
print(type(z))

## Indexing

In [0]:
z[0, :] = 0
z[:, 2] = 1
print(z)

In [0]:
z[:,:] = 0.5
print(z)

# Simple Neural Network on CIFAR-10

## Dataloader Definition

Define variables that will be needed for loading data.

In [0]:
batch_size = 4
num_workers = 2
root_dir = './data'
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Define the preprocessing that will be done with each image before loading.

In [0]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

Define the train and test dataset.

In [0]:
trainset = torchvision.datasets.CIFAR10(root=root_dir, train=True,
                                        download=True, transform=transform)

testset = torchvision.datasets.CIFAR10(root=root_dir, train=False,
                                       download=True, transform=transform)

Define the dataloaders for both the train dataset and test dataset.

In [0]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=num_workers)

testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=num_workers)

### Visualizing a single batch from the dataloader

In [0]:
def imshow(img):
  img = img / 2 + 0.5
  npimg = img.numpy()
  plt.imshow(np.transpose(npimg, (1, 2, 0)))
  plt.show()

In [0]:
dataiter = iter(trainloader)
images, labels = dataiter.next()

imshow(torchvision.utils.make_grid(images))

In [0]:
torchvision.utils.make_grid(images).size()

### Making custom transforms, datasets and dataloaders

You can make custom:

* transforms (preprocessing)
* datasets (loading datasets from various places and in various forms)
* dataloaders (accumulating loaded data in a particular way)

Have a look at the following tutorial by Pytorch for more details:
[Tutorial](https://pytorch.org/tutorials/beginner/data_loading_tutorial.html)

## Model Definition

While defining the model, make sure the model class extends `nn.Module`.

When `nn.Module` is extended, it requires two functions to be defined:
* `__init__()`
* `forward()`

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

Initialize the model and transfer it to the device you will be using.

In [0]:
net = Net().to(device)

The `DataParallel()` function is used to parallelize the training process across multiple GPUs.

In [0]:
net = nn.DataParallel(net)

## Training Definition

Define the relevant variables for training the model.

In [0]:
n_epochs = 2
model_ckpt = './cifar_net.pth'
learning_rate = 0.001
momentum = 0.9

Define the optimizer for training the model.

In [0]:
optimizer = optim.SGD(net.parameters(), lr = learning_rate, momentum = momentum)

Define the loss function to be used.

In [0]:
criterion = nn.CrossEntropyLoss()

Define the training procedure!

In [0]:
for epoch in range(n_epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1}] Loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

Save the model weights.

In [0]:
torch.save(net.state_dict(), model_ckpt)

## Test the Model
 

Show test images and corresponding ground truth.

In [0]:
dataiter = iter(testloader)
images, labels = dataiter.next()

imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(4)))

Show predicted classes by the model.

In [0]:
outputs = net(images)

_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join('%5s' % classes[predicted[j]]
                              for j in range(4)))

Calculate accuracy of the model.

In [0]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images.to(device))
        _, predicted = torch.max(outputs.cpu().data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

# Tensorboard